# TA with ChatGPT
_2023-12-10_

https://medium.com/@kokhua81/stocks-technical-analysis-ta-with-python-chatgpt-a-comprehensive-guide-871a756ebc7c

## Overview

### TA Steps

1. Utilize yfinance to fetch historical stock price data and [pandas_ta](https://github.com/twopirllc/pandas-ta) to compute technical indicators.
1. Gather indicators for the last trading day to input into the ChatGPT prompt.
1. Create a prompt for ChatGPT, incorporating the indicators for analysis.
1. Engage ChatGPT to interpret indicators and predict future trends.
1. Enhance insights by visualizing indicator trends using additional charts.

## Trial Run

In [ ]:
!pip install yfinance
!pip install pandas_ta

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import matplotlib.dates as mdates
from datetime import datetime, timedelta

### Data

In [ ]:
# Define the stock symbol and timeframe
symbol = 'DIS'
end_date = datetime.today()
start_date = end_date - timedelta(days=120)  # 4 months before today

# Fetch stock data using yfinance
stock_data = yf.download(symbol, start=start_date, end=end_date)

# Calculate technical indicators using pandas-ta
stock_data.ta.macd(append=True)
stock_data.ta.rsi(append=True)
stock_data.ta.bbands(append=True)
stock_data.ta.obv(append=True)

# Calculate additional technical indicators
stock_data.ta.sma(length=20, append=True)
stock_data.ta.ema(length=50, append=True)
stock_data.ta.stoch(append=True)
stock_data.ta.adx(append=True)

# Calculate other indicators
stock_data.ta.willr(append=True)
stock_data.ta.cmf(append=True)
stock_data.ta.psar(append=True)

#convert OBV to million
stock_data['OBV_in_million'] =  stock_data['OBV']/1e7
stock_data['MACD_histogram_12_26_9'] =  stock_data['MACDh_12_26_9'] # not to confuse chatGTP

# Summarize technical indicators for the last day
last_day_summary = stock_data.iloc[-1][['Adj Close',
    'MACD_12_26_9','MACD_histogram_12_26_9', 'RSI_14', 'BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0','SMA_20', 'EMA_50','OBV_in_million', 'STOCHk_14_3_3', 
    'STOCHd_14_3_3', 'ADX_14',  'WILLR_14', 'CMF_20', 
    'PSARl_0.02_0.2', 'PSARs_0.02_0.2'
]]

print("Summary of Technical Indicators for the Last Day:")
print(last_day_summary)

### Prompt

Construct the prompt in a manner that invokes the expertise of legendary TA figures like Charles Dow and John Bollinger to conduct the analysis on your behalf.

Below is the Python script that establishes the prompt, delineating ChatGPT’s role and integrating the most recent trading day’s indicators. Following ChatGPT’s analysis, you can further engage by posing additional questions, delving into specific indicators, and requesting ChatGPT to elaborate further. Do note instead of pasting the prompt to ChatGPT manually, you can also use the openai API to automate the process.

In [ ]:
sys_prompt = """
Assume the role as a leading Technical Analysis (TA) expert in the stock market, \
a modern counterpart to Charles Dow, John Bollinger, and Alan Andrews. \
Your mastery encompasses both stock fundamentals and intricate technical indicators. \
You possess the ability to decode complex market dynamics, \
providing clear insights and recommendations backed by a thorough understanding of interrelated factors. \
Your expertise extends to practical tools like the pandas_ta module, \
allowing you to navigate data intricacies with ease. \
As a TA authority, your role is to decipher market trends, make informed predictions, and offer valuable perspectives.

given {} TA data as below on the last trading day, what will be the next few days possible stock price movement? 

Summary of Technical Indicators for the Last Day:
{}""".format(symbol,last_day_summary)

print(sys_prompt)

### Insight

# Plot the technical indicators
plt.figure(figsize=(14, 8))

# Price Trend Chart
plt.subplot(3, 3, 1)
plt.plot(stock_data.index, stock_data['Adj Close'], label='Adj Close', color='blue')
plt.plot(stock_data.index, stock_data['EMA_50'], label='EMA 50', color='green')
plt.plot(stock_data.index, stock_data['SMA_20'], label='SMA_20', color='orange')
plt.title("Price Trend")
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))  # Format date as "Jun14"
plt.xticks(rotation=45, fontsize=8)  # Adjust font size
plt.legend()

# On-Balance Volume Chart
plt.subplot(3, 3, 2)
plt.plot(stock_data['OBV'], label='On-Balance Volume')
plt.title('On-Balance Volume (OBV) Indicator')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))  # Format date as "Jun14"
plt.xticks(rotation=45, fontsize=8)  # Adjust font size
plt.legend()

# MACD Plot
plt.subplot(3, 3, 3)
plt.plot(stock_data['MACD_12_26_9'], label='MACD')
plt.plot(stock_data['MACDh_12_26_9'], label='MACD Histogram')
plt.title('MACD Indicator')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))  # Format date as "Jun14"
plt.xticks(rotation=45, fontsize=8)  # Adjust font size
plt.title("MACD")
plt.legend()

# RSI Plot
plt.subplot(3, 3, 4)
plt.plot(stock_data['RSI_14'], label='RSI')
plt.axhline(y=70, color='r', linestyle='--', label='Overbought (70)')
plt.axhline(y=30, color='g', linestyle='--', label='Oversold (30)')
plt.legend()
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))  # Format date as "Jun14"
plt.xticks(rotation=45, fontsize=8)  # Adjust font size
plt.title('RSI Indicator')

# Bollinger Bands Plot
plt.subplot(3, 3, 5)
plt.plot(stock_data.index, stock_data['BBU_5_2.0'], label='Upper BB')
plt.plot(stock_data.index, stock_data['BBM_5_2.0'], label='Middle BB')
plt.plot(stock_data.index, stock_data['BBL_5_2.0'], label='Lower BB')
plt.plot(stock_data.index, stock_data['Adj Close'], label='Adj Close', color='brown')
plt.title("Bollinger Bands")
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))  # Format date as "Jun14"
plt.xticks(rotation=45, fontsize=8)  # Adjust font size
plt.legend()

# Stochastic Oscillator Plot
plt.subplot(3, 3, 6)
plt.plot(stock_data.index, stock_data['STOCHk_14_3_3'], label='Stoch %K')
plt.plot(stock_data.index, stock_data['STOCHd_14_3_3'], label='Stoch %D')
plt.title("Stochastic Oscillator")
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))  # Format date as "Jun14"
plt.xticks(rotation=45, fontsize=8)  # Adjust font size
plt.legend()

# Williams %R Plot
plt.subplot(3, 3, 7)
plt.plot(stock_data.index, stock_data['WILLR_14'])
plt.title("Williams %R")
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))  # Format date as "Jun14"
plt.xticks(rotation=45, fontsize=8)  # Adjust font size

# ADX Plot
plt.subplot(3, 3, 8)
plt.plot(stock_data.index, stock_data['ADX_14'])
plt.title("Average Directional Index (ADX)")
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))  # Format date as "Jun14"
plt.xticks(rotation=45, fontsize=8)  # Adjust font size

# CMF Plot
plt.subplot(3, 3, 9)
plt.plot(stock_data.index, stock_data['CMF_20'])
plt.title("Chaikin Money Flow (CMF)")
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))  # Format date as "Jun14"
plt.xticks(rotation=45, fontsize=8)  # Adjust font size

# Show the plots
plt.tight_layout()
plt.show()

## Sentiment Analysis from Options Data

One intriguing avenue for gaining insights into market sentiment is through the analysis of options data. Options, which offer the right to buy or sell a stock at a specific price within a predetermined timeframe, can reveal valuable clues about investor expectations and market trends.

By examining key metrics derived from options data, such as implied volatility, open interest, and put-call ratios, investors can gauge market sentiment and anticipate potential price movements. Elevated levels of implied volatility, for instance, may indicate increased uncertainty and potential price swings, while shifts in put-call ratios can provide insights into prevailing bullish or bearish sentiments.

### Python to get options data, calculate metrics
In this tutorial, we utilize Python and related libraries to retrieve options data and calculate essential metrics like implied volatility, open interest, and put-call ratios. These derived metrics provide insights into stock sentiment, both in the short term and over a longer horizon, considering option expiration dates.

Below are the key steps for the python script

Import libraries (yahoo_fin) and fetch current stock price and option expiry dates.
Retrieve call and put option data for the nearest expiry date.
Analyze options data for implied volatility, option prices, strike prices, volume, and open interest.
Calculate put-call ratio, implied volatility percentiles, and skew.

In [ ]:
from yahoo_fin import options, stock_info
import pandas as pd
from IPython.utils import io
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
ticker = 'NVDA'
current_price = round(stock_info.get_live_price(ticker),3)
option_expiry_dates = options.get_expiration_dates(ticker)# get list of expiry date
nearest_expiry = option_expiry_dates[0] # can vary the list of expiry date

call_df = options.get_calls(ticker, nearest_expiry)
put_df = options.get_puts(ticker, nearest_expiry)

# Capture the print statement in Google Colab
with io.capture_output() as captured:
    # Implied Volatility Analysis:
    avg_call_iv = call_df["Implied Volatility"].str.rstrip("%").astype(float).mean()
    print("\nAverage Implied Volatility for Call Options:", avg_call_iv, "%")

    avg_put_iv = put_df["Implied Volatility"].str.rstrip("%").astype(float).mean()
    print("Average Implied Volatility for Put Options:", avg_put_iv, "%")

    # Option Prices Analysis:
    avg_call_last_price = call_df["Last Price"].mean()
    print("\nAverage Last Price for Call Options:", avg_call_last_price)

    avg_put_last_price = put_df["Last Price"].mean()
    print("Average Last Price for Put Options:", avg_put_last_price)

    # Strike Price Analysis:
    min_call_strike = call_df["Strike"].min()
    max_call_strike = call_df["Strike"].max()
    print("\nMinimum Strike Price for Call Options:", min_call_strike)
    print("Maximum Strike Price for Call Options:", max_call_strike)

    min_put_strike = put_df["Strike"].min()
    max_put_strike = put_df["Strike"].max()
    print("Minimum Strike Price for Put Options:", min_put_strike)
    print("Maximum Strike Price for Put Options:", max_put_strike)

    # Volume Analysis:
    total_call_volume =  call_df["Volume"].str.replace('-','0').astype(float).sum()
    print("\nTotal Volume for Call Options:", total_call_volume)

    try:
        total_put_volume =  put_df["Volume"].str.replace('-','0').astype(float).sum()
    except:
        total_put_volume =  put_df["Volume"].sum()

    print("Total Volume for Put Options:", total_put_volume)

    # Open Interest Analysis:
    try:
        call_df['Open Interest'] = call_df['Open Interest'].str.replace('-','0').astype(float)
    except:
        pass
    try:
        put_df['Open Interest'] = put_df['Open Interest'].str.replace('-','0').astype(float)
    except:
        pass
    total_call_open_interest = call_df["Open Interest"].sum()
    print("\nTotal Open Interest for Call Options:", total_call_open_interest)

    total_put_open_interest = put_df["Open Interest"].sum()
    print("Total Open Interest for Put Options:", total_put_open_interest)
    try:
        call_df['Implied Volatility'] = call_df['Implied Volatility'].str.replace('%','').astype(float)
    except:
        pass

    try:
        put_df['Implied Volatility'] = put_df['Implied Volatility'].str.replace('%','').astype(float)
    except:
        pass
    # Calculate Option Greeks (Delta, Gamma, Theta, Vega, and Rho) - You can use appropriate formulas to calculate these.

    # Calculate Put-Call Ratio
    put_call_ratio = total_put_volume / total_call_volume

    # Calculate Implied Volatility Percentile
    call_iv_percentile = (call_df['Implied Volatility'] > call_df['Implied Volatility'].mean()).mean() * 100
    put_iv_percentile = (put_df['Implied Volatility'] > put_df['Implied Volatility'].mean()).mean() * 100

    # Calculate Implied Volatility Skew
    implied_vol_skew = call_df['Implied Volatility'].mean() - put_df['Implied Volatility'].mean()

    # Check if call options have higher average implied volatility than put options
    is_bullish_sentiment = call_df['Implied Volatility'].mean() > put_df['Implied Volatility'].mean()

    # Display the results
    print("Put-Call Ratio:", put_call_ratio)
    print("Call Option Implied Volatility Percentile:", call_iv_percentile)
    print("Put Option Implied Volatility Percentile:", put_iv_percentile)
    print("Implied Volatility Skew:", implied_vol_skew)

    # if is_bullish_sentiment:
    #     print(f"The overall sentiment of {ticker} is bullish.")
    # else:
    #     print(f"The overall sentiment of {ticker}  is bearish.")

# Get the captured output as a string
captured_output = captured.stdout

# Print the captured output or save it to a variable or file as needed
print(captured_output)

### Analyze data with ChatGPT

In [ ]:
role_prompt = """Assume the role as a seasoned stock option analyst with a strong track record\
 in dissecting intricate option data to discern valuable insights into stock sentiment.\
  Proficient in utilizing advanced statistical models and data visualization techniques to \
  forecast market trends and make informed trading decisions. Adept at interpreting option Greeks, \
  implied volatility, and analyzing trading volumes to gauge investor sentiment accurately.\
   Known for an exceptional ability to transform complex data into actionable trading strategies, \
   consistently achieving optimal results for portfolio growth."""

today_date = datetime.now().strftime("%b %d, %Y")

advice_prompt = f"Given {ticker} current {today_date} price is {current_price} and \
call and put option at expiry date {nearest_expiry} is as below: \n{captured_output}\n\
What is the overall sentiment of {ticker} based on above information?"
print(role_prompt + '\n' + advice_prompt)